In [1]:
import numpy as np 
import pandas as pd 
import itertools
import cv2
import tensorflow as tf 

import keras.backend as K
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt
import os

### Загрузка моделей

In [2]:
base_model=MobileNetV2(input_shape=[256, 256, 3], include_top=False)

## Слои

In [3]:
# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
base_model_outputs = [base_model.get_layer(name).output for name in layer_names]
# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)
down_stack.trainable = False

## Модель функции

In [4]:
def upsample(filters, size, norm_type='batchnorm', apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(
      tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                      padding='same',
                                      kernel_initializer=initializer,
                                      use_bias=False))

    if norm_type.lower() == 'batchnorm':
        result.add(tf.keras.layers.BatchNormalization())
    elif norm_type.lower() == 'instancenorm':
        result.add(InstanceNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

        result.add(tf.keras.layers.ReLU())

    return result


up_stack = [
    upsample(512, 3),  # 4x4 -> 8x8
    upsample(256, 3),  # 8x8 -> 16x16
    upsample(128, 3),  # 16x16 -> 32x32
    upsample(64, 3),   # 32x32 -> 64x64
]

def unet_model(output_channels):
    inputs = tf.keras.layers.Input(shape=[256, 256, 3])

    # Downsampling through the model
    skips = down_stack(inputs)
    x = skips[-1]
    skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = tf.keras.layers.Concatenate()
        x = concat([x, skip])

  # This is the last layer of the model
    last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2, activation='sigmoid',
      padding='same')  #64x64 -> 128x128

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

In [53]:
model = unet_model(1)
model.load_weights('Unet-disk-model.h5')

def get_rectangle(image):
    global model  # чтобы каждый раз во время тестирования не подгружать модель - начинает выкидывать warning-и
    car_image = tf.image.resize(image, [256,256])
    car_image = tf.cast(car_image, tf.float32) / 255.0  
    pred_mask = model.predict(car_image[tf.newaxis, ...],verbose=1,steps=1)
    pred_mask=pred_mask.reshape(256,256,1)   
    new_img=tf.keras.preprocessing.image.array_to_img(pred_mask)
    binary = np.array(new_img)
    contours, hierarchy =  cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(contours, key = cv2.contourArea, reverse = True)[:2]
    for c in cnts:
        perimeter = cv2.arcLength(c, True)
        area = cv2.contourArea(c)
        if area<5*10**4:
            accuracy = 0.001 * cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, accuracy, True)
    minx,maxx,miny,maxy=260,0,260,0
    for c_ in c:
        minx=min(minx,c_[:,0])
        miny=min(miny,c_[:,1])
        maxx=max(maxx,c_[:,0])
        maxy=max(maxy,c_[:,1])
    return [minx,maxx,miny,maxy]   
    

def func_to_test(image, coords, dim):
    
    image=cv2.imread('C:/Users/leh-1/Documents/TrainYourOwnYOLO/Data/Source_Images/Test_Images/' + image)
    
    if dim == 1:
        first_tire = coords[0]
        rimage1 = image[first_tire[2]:first_tire[3],first_tire[0]:first_tire[1]]
        return [get_rectangle(rimage1)]
    
    if dim == 2:
        first_tire = coords[0]
        second_tire = coords[1]
        rimage1 = image[first_tire[2]:first_tire[3],first_tire[0]:first_tire[1]]
        rimage2 = image[second_tire[2]:second_tire[3],second_tire[0]:second_tire[1]]
        return [get_rectangle(rimage1), get_rectangle(rimage2)]

In [54]:
import pytest
import ipytest

ipytest.config.rewrite_asserts = True

In [60]:
# выгрузка csv, полученной от YOLO
csv = pd.read_csv('C:/Users/leh-1/Documents/TrainYourOwnYOLO/3_Inference/out.csv')
df = csv[['image', 'coords']]
df.loc[:]['image'] = df['image'].str.split('\\').str.get(-1)

In [61]:
@pytest.mark.parametrize("img", df['image'].to_list())
def test_func_to_test(img):
    
    global df
    
    dimension = df[df['image'] == img]['coords'].to_list()[0].count('[') - 1  # проверка количества найденных YOLO колёс
    
    if dimension == 0:
        assert True
        return None
    
    if dimension == 1:
        list_raw = df[df['image'] == img]['coords'].to_list()[0].replace('[', '').replace(']','').replace(' ', '').split(',')
        l_left = []
        for num in list_raw[0:4]:
            l_left.append(int(num))
        coordinates = [l_left]
    
    if dimension == 2:
        l_left = []
        l_right = []
        list_raw = df[df['image'] == img]['coords'].to_list()[0].replace('[', '').replace(']','').replace(' ', '').split(',')
        for num in list_raw[0:4]:
            l_left.append(int(num))
        for num in list_raw[4:8]:
            l_right.append(int(num))
        
        coordinates = [l_left, l_right]

    List_out = func_to_test(img, coordinates, dimension)
    assert len(List_out) == dimension
    
    #list_first_wheel = []
    #list_second_wheel = []
    #for i in range(4):
    #    list_first_wheel.append(l1[i][0])
    #    list_second_wheel.append(l2[i][0])
    final_dict[img] = List_out
# тест проверяет, что итогом работы функции стало то количество спиское, что и было на выходе YOLO
# и каждый из них содержит 4 значения

In [62]:
# точка - удачный тест, 'F' - тест закончился ошибкой
final_dict = {}
ipytest.run('-qq')

..................................................                                           [100%]


In [63]:
final_dict

{'00021.jpg': [[array([0], dtype=int32),
   array([255], dtype=int32),
   array([208], dtype=int32),
   array([255], dtype=int32)],
  [array([0], dtype=int32),
   array([109], dtype=int32),
   array([237], dtype=int32),
   array([255], dtype=int32)]],
 '00022.jpg': [[array([18], dtype=int32),
   array([67], dtype=int32),
   array([79], dtype=int32),
   array([153], dtype=int32)],
  [array([223], dtype=int32),
   array([255], dtype=int32),
   array([23], dtype=int32),
   array([57], dtype=int32)]],
 '00027.jpg': [[array([145], dtype=int32),
   array([193], dtype=int32),
   array([32], dtype=int32),
   array([43], dtype=int32)],
  [array([73], dtype=int32),
   array([165], dtype=int32),
   array([215], dtype=int32),
   array([233], dtype=int32)]],
 '00033.jpg': [[array([139], dtype=int32),
   array([239], dtype=int32),
   array([7], dtype=int32),
   array([75], dtype=int32)],
  [array([175], dtype=int32),
   array([207], dtype=int32),
   array([245], dtype=int32),
   array([255], dtype=i

In [46]:
B = [4, 6, 8, 3]
b for b in B

SyntaxError: invalid syntax (<ipython-input-46-070fbc7c36b2>, line 2)